# Demo 1 - CNN-based image classification using CIFAR-10

##**Scenario:**
You are building an image classification system for a company that wants to automatically tag images into categories like airplane, car, bird, cat, etc. The dataset provided is CIFAR-10 (10 classes, small RGB images).

##**Objective:**
To train a CNN model on CIFAR-10 while applying model optimization techniques like learning rate scheduling, normalization, regularization, data augmentation, and visualize training using TensorBoard.



## Step 1: Import Required Libraries
 Import all required libraries for model building, training, and visualization.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import os

##  Step 2: Load and Preprocess the CIFAR-10 Dataset
Load CIFAR-10, normalize pixel values, and one-hot encode labels.



In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

## Step 3: Set Up Data Augmentation
Use ImageDataGenerator to augment training data to improve generalization.

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)

## Step 4: Define the CNN Model with Regularization and Batch Normalization
Build a CNN with dropout, L2 regularization, and batch normalization to combat overfitting and improve training.

In [ ]:
def build_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu',
                            kernel_regularizer=regularizers.l2(0.001), input_shape=(32, 32, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.25))

    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.25))

    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax'))

    return model

model = build_model()

## Step 5: Compile the Model with Optimizer and Learning Rate Scheduling
Compile the model with Adam optimizer, gradient clipping, and add learning rate scheduler to reduce LR on plateau.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

## Step 6: Setup TensorBoard and ModelCheckpoint
Enable TensorBoard for visualizing training progress and add checkpoint to save best model.

In [ ]:
log_dir = "logs/demo1"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1)

## Step 7: Train the Model with Augmentation
Train the model using augmented data with TensorBoard, checkpointing, and LR scheduling.

In [ ]:
history = model.fit(datagen.flow(x_train, y_train, batch_size=64),
                    epochs=2,
                    validation_data=(x_test, y_test),
                    callbacks=[tensorboard_callback, checkpoint, lr_scheduler])

## Step 8: Launch TensorBoard (from terminal)
Running this in terminal to start TensorBoard and view training curves, gradients, histograms, and model graph.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/demo1